In [1]:
%cd ../src
%load_ext autoreload
%autoreload 2

from credibility import claim_or_opinion
import json
import pandas as pd
from IPython.display import display, HTML

df = pd.read_csv('/home/daan/ws/tide/data/Russian_border_Ukraine.csv')
print = lambda tweet: display(HTML(tweet))

/home/daan/ws/tide/team-tulip-troops/pipeline/src


/home/daan/miniconda3/envs/tuliptroops/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tweet1 = df.loc[46265,'renderedContent']

In [3]:
tweet2 = df.loc[552,'renderedContent']

In [4]:
print(tweet1)
claim_or_opinion(tweet1)

('statement', 0.9215100407600403)

In [5]:
print(tweet2)
claim_or_opinion(tweet2)

('opinion', 0.6656166315078735)

In [9]:
from credibility import zero_shot_model
from credibility import initialize_nli
from credibility import collect_evidences

def parse_conclusion(conclusion):
    return dict(zip(map(lambda l: l.split(':')[0],conclusion['labels']),conclusion['scores']))

In [10]:
tweet3 = df.loc[51886,'renderedContent'].replace("\n"," ")
print(tweet3)
claim_or_opinion(tweet3)

('statement', 0.8697166442871094)

In [11]:
evidence = collect_evidences(tweet3,datasource='elastic')
evidence_text = evidence.loc[6].to_dict()['summary']

In [12]:
print("Tweet: "+ tweet3)
print("Evidence: "+ evidence_text)
zeroshot_labels = [f"The following statement is True: {tweet3}",
                   f"Not enough information to verify the following statement: {tweet3}",
                   f"The following statement is False: {tweet3}"]
conclusion = zero_shot_model(evidence_text, candidate_labels=zeroshot_labels)
parse_conclusion(conclusion)

{'The following statement is True': 0.47842562198638916,
 'Not enough information to verify the following statement': 0.2614210844039917,
 'The following statement is False': 0.26015326380729675}

In [13]:
tweet4 = df.loc[12866,'renderedContent'].replace("\n"," ")
print(tweet4)
claim_or_opinion(tweet4)

('statement', 0.8970088958740234)

In [14]:
evidence = collect_evidences(tweet4,datasource='elastic')
evidence_text = evidence.loc[6].to_dict()['summary']

In [15]:
print("Tweet: "+ tweet4)
print("")
print("Evidence: "+ evidence_text)
zeroshot_labels = [f"The following statement is True: {tweet4}",
                   f"Not enough information to verify the following statement: {tweet4}",
                   f"The following statement is False: {tweet4}"]
conclusion = zero_shot_model(evidence_text, candidate_labels=zeroshot_labels)
parse_conclusion(conclusion)

{'The following statement is False': 0.5344493389129639,
 'Not enough information to verify the following statement': 0.34265515208244324,
 'The following statement is True': 0.12289545685052872}

In [16]:
from urllib.parse import urlparse

In [24]:
def get_clean_source(source):
    if source.startswith("http"):
        return '.'.join(urlparse(source).netloc.split('.')[-2:-1])
    elif " - " in source:
        return source.split(' - ')[0]

In [26]:
source_url = "https://www.bbc.com/news/world-europe-60534959"
source_name = get_clean_source(source_url)

zeroshot_labels = ['credible source', 'uncredible source']
conclusion = zero_shot_model(source_name, candidate_labels=zeroshot_labels)

source_name, parse_conclusion(conclusion)

('bbc',
 {'credible source': 0.8198277354240417,
  'uncredible source': 0.18017227947711945})

In [30]:
source_url = "https://www.bbc.com/news/world-europe-60534959"
source_name = get_clean_source(source_url)

zeroshot_labels = ['british affiliation', 'russian affiliaion', 'canadian affiliaion']
conclusion = zero_shot_model(source_name, candidate_labels=zeroshot_labels)

source_name, parse_conclusion(conclusion)

('bbc',
 {'british affiliation': 0.9695330858230591,
  'canadian affiliaion': 0.019143691286444664,
  'russian affiliaion': 0.01132317166775465})

In [54]:
source_url = "https://news-front.info/2023/02/22/doroga-zhizni-kak-zhivut-zhiteli-odnogo-iz-razrushennyh-poselkov-v-hersonskoj-oblasti/"
source_url = "https://www.infowars.com/posts/alex-jones-backs-trump-after-anti-war-speech/"
#source_url = "https://www.bbc.com/news/world-europe-60534959"
source_name = get_clean_source(source_url)

zeroshot_labels = ['misinformation', 'credible source']
zeroshot_labels = ['credible source', 'uncredible source']
conclusion = zero_shot_model(source_name, candidate_labels=zeroshot_labels)

#zeroshot_labels = ['Russian affiliation', 'no Russian affiliation']
#conclusion = zero_shot_model(source_name, candidate_labels=zeroshot_labels)

source_name, parse_conclusion(conclusion)

('infowars',
 {'uncredible source': 0.8476134538650513,
  'credible source': 0.15238656103610992})

In [96]:
parsed_conclusion = conclusion['labels'][2].split(' ')[4].lower().replace(':', '')
parsed_conclusion

'verify'

In [47]:
nli_model = initialize_nli()
scores = nli_model.predict([(tweet3, evidence_text)])

In [64]:
unsure_threshold = 0.4
evidence_dict = {}
label_mapping = ['false', 'true', 'neutral']
confidence = scores[0,scores.argmax(axis=1)[0]]
label = 'neutral' if confidence <= unsure_threshold else label_mapping[scores.argmax(axis=1)[0]]
result = {'label':label,'confidence':confidence}
evidence_dict.update(result)

In [65]:
scores[0,2]

0.6832002

In [67]:
evidence_dict

{'label': 'neutral', 'confidence': 0.6832002}

In [54]:
dict(zip(['false', 'true', 'neutral'],list(scores[0])))

{'false': -0.005442068, 'true': -1.0628991, 'neutral': 0.6832002}

In [43]:
evidence

,Unnamed: 0,source,text,texthash,summary
0,0,Nyt - 2022-05-31T00:00:00 - U.S. to Send Ukrai...,U.S. to Send Ukraine $700 Million in Military ...,ba8b57792c2814b905740b9b54f0c628,"Biden: ""Any use of nuclear weapons in this co..."
1,1,The Guardian - 2022-07-31T06:00:47Z - Ukrainia...,Ukrainian offensive forces Russia to bolster t...,7466404018a9fde0122a2acba741b500,Russian troop movements come in response to U...
2,2,Nyt - 2022-06-01T00:00:00 - Two Telling Numbers,"Two Telling Numbers In the Opinion section, P...",eed93cc82b84d99319c9aa8370b7fcf5,The big question is whether Russia can encirc...
3,3,Nyt - 2022-07-27T00:00:00 - Special Military C...,Special Military Cell Flows Weapons and Equipm...,ad745320a1e9e12c5943349028d496a2,About 75 percent of the arms are sent to stag...
4,4,The Guardian - 2022-02-24T19:54:49Z - War in U...,War in Ukraine: where has Russia attacked? Ru...,b7d8e3cc1feeeb14325a8a3f86c7e633,Russia's attack on Ukraine began at about 5am...
5,5,The Guardian - 2022-07-08T15:58:40Z - Germany ...,Germany to reactivate coal power plants as Rus...,204013a47046f4fe0727b61a309fa229,Germany to reactivate mothballed coal-fired p...
6,6,The Guardian - 2022-05-02T04:42:11Z - Russia-U...,Russia-Ukraine war: South Korea set to reopen ...,a42d76f869eeca13c163b2f5b678193f,Around 100 civilians evacuated from Azovstal ...
7,7,The Guardian - 2022-07-19T12:28:54Z - Ukrainia...,Ukrainian boy held hostage by Russia tells of ...,70561f9b29ee69b909d480d9824e9d58,"Vladislav Buryak, 16, was separated from his ..."
8,8,Nyt - 2022-06-01T00:00:00 - Putin’s Threats Hi...,Putin’s Threats Highlight the Dangers of a New...,3f089af740049f18bbaf5c6469f61f5b,Biden: Any use of nuclear weapons in Ukraine ...
9,9,Nyt - 2022-07-27T00:00:00 - The U.S. and Russi...,The U.S. and Russia Need to Start Talking Befo...,46617db584a1a91681b08ab519d901bc,The U.S. and Russia need to start talking bef...
